# Cian flats EDA

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

# USE THIS STYLE
# plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle')
# 
# OR THIS STYLE
import aquarel

theme = aquarel.load_theme("arctic_light")
theme.set_font(family="serif")
theme.apply()

# Сделаем автоподгрузку всех изменений при перепрогонке ячейки
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
custom_pallete = {
    "red": "#BF616A",
    "orange": "#D08770",
    "yellow": "#EBCB8B",
    "green": "#A3BE8C",
    "purple": "#B48EAD",
    "light_green": "#8FBCBB",
    "light_gray_blue": "#88C0D0",
    "light_blue": "#81A1C1",
    "blue": "#5E81AC",
}

## Load data

In [19]:
df_cian = pd.read_csv("./data/cian_houses_dataset.csv")
df_cian.head()

,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count
0,18374400,"Москва, СВАО, р-н Марьина роща, Тэйт жилой ком...",55.802530,37.620945,Марьина Роща,34,49,38.28,11.8,unknown,...,3.12,unknown,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,КОРТРОС,https://www.cian.ru/sale/flat/302789697/,1
1,49954800,"Москва, ЦАО, р-н Таганский, Славянская пл., 2/5с1",55.752541,37.635727,Китай,7,8,53.20,unknown,24.1,...,3.0,unknown,Монолитно-кирпичный,unknown,unknown,unknown,Вторичка / Апартаменты,unknown,https://www.cian.ru/sale/flat/301496936/,1
2,8170000,"Москва, НАО (Новомосковский), Филимонковское п...",55.552637,37.337172,Филатов Луг,2,5,31.00,unknown,10.0,...,3.0,Во двор,unknown,Нет информации,Нет,unknown,Вторичка,unknown,https://www.cian.ru/sale/flat/303552909/,1
3,20206500,"Москва, СВАО, р-н Останкинский, ул. Годовикова...",55.810466,37.624247,Алексеевская,34,37,28.50,21.4,unknown,...,3.01,На улицу,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,КОРТРОС,https://www.cian.ru/sale/flat/306568424/,1
4,17119620,"Москва, ЮАО, р-н Даниловский, Автозаводская ул...",55.706597,37.632285,Тульская,6,20,38.82,10.6,18.6,...,2.85,Во двор,Монолитный,unknown,unknown,unknown,Новостройка,unknown,https://www.cian.ru/sale/flat/305679800/,1


In [20]:
df_cian.shape

(8053, 22)

Разделим на численные и категориальные фитчи

In [22]:
numerical_features = [f for f in df_cian.columns if df_cian.dtypes[f] != 'object']
categorical_features = [f for f in df_cian.columns if df_cian.dtypes[f] == 'object']

In [23]:
numerical_features

['price', 'geo_lat', 'geo_lng', 'floor', 'floor_count', 'square', 'room_count']